# 文本分类与打标签

本章内容

1. 本章内容概述
2. 基于卷积神经网络的文本分类模型详解
3. 基于RNN/LSTM的文本分类模型详解
4. 构建卷积神经网络完成影评褒贬分类
5. 构建LSTM模型完成新闻文本分类

## 本章内容概述

文本分类是一个常见的人工智能任务。通常我们需要对

## 基于卷积神经网络的文本分类模型

在这一节里，我们将介绍如何使用卷积神经网络（CNN）模型进行文本分类。

- 我们会首先介绍卷积神经网络的原理及相关的概念，比如池化等。
- 其次我们介绍一维卷积网络在文字上的应用。
- 随后，通过与图像数据的对比引入如何将二维卷积神经网络方法运用到文本信息建模中。
- 最后，我们将拓展单通道二维卷积神经网络到多通道卷积神经网络模型。

### 什么是卷积神经网络
卷积神经网络模型是利用卷积滤子对输入数据进行分析的技术。在数学里，卷积一词的含义是指通过两个函数f和g生成第三个函数h的操作，产出函数f经过函数g变换后的形态。通常函数g会对原始函数进行平移或者翻转操作。卷积既指代这个操作，也指代这个操作的结果函数。卷积还可以被看作是“加权移动平均”的推广。 我们利用下面这个wiki的图来演示卷积操作。
<img src="./pics/Convolution3.png" width="450">
这个图中有五行图片。
- 第一行的两幅图分别是原始函数f(t)和g(t)。假设两个函数都用 ${\tau }$ 来表示，从而得到f( $\tau$ )和g( $ \tau$ )。
- 第二行的图片希纳是将函数g($\tau$)向右移动t个单位并翻转后得到函数g( $\tau -t$)的图像。将g( $\tau -t$)翻转至纵轴另一侧，得到g(-($\tau$  -t))即g(t- $\tau$ )的图像。
- 第三行的图片显示了当时间变量$\tau$取不同值时，函数g(t-$\tau$ ) 沿着时间轴$\tau$“滑动”的动作。
- 第四行的图片显示了当f(t)和g(t)两函数交会时,两函数乘积的积分值。这个积分值可以理解为使用g(t-$\tau$)做为加权函数，来对f($\tau$)取加权值的结果。
- 第五行的图片显示了这个加权平均的持续过程。

最后得到的加权平均的结果就是f和g的卷积。如果f（t）是一个单位脉冲，我们得到的乘积就是g（t）本身，称为冲激响应。

<img src="./pics/Convolution_self.gif" width="450">
以上来自wiki的图显示了两个方形脉冲波的卷积。其中
函数"g"首先对 $\tau =0$反射，接着平移"t"，成为 $ g(t-\tau )$。那么重叠部分的面积就相当于"t"处的卷积，其中横坐标代表待变量 $ \tau $以及新函数 $ f\ast g$的自变量"t"。

<img src="./pics/Convolution_spiky.gif" width="450">
以上来自wiki的图显示了方形脉冲波和指数衰退的脉冲波的卷积，同样地重叠部分面积就相当于"t"处的卷积。注意到因为"g"是对称的，所以在这两张图中，反射并不会改变它的形状。

以上是对一维卷积的解释。卷积神经网络最常见于图像识别中，因此通常是使用二维卷积。下面简要说明。
<img src="./pics/CNNconcept.png" width="500">

在一个二维卷积网络里面

### 使用一维卷积神经网络进行文本分类模型

从《词嵌入与词向量》这节课中我们了解到可以将一组文本中的任意单词映射到一个指定维度的空间。这个指定的维度通常比文本中全部单词所需要的维度低很多，但是通常仍然需要数百维。我们需要将使用卷积神经网络将这数百维的向量再次压缩到更低的向量中。

### 使用二维卷积神经网络进行文本分类模型

我们也可以使用二维卷积神经网络对文本建模。这是因为，我们可以将一个文本中相邻的单词对应的词向量叠加起来，那么对于任意给定的窗口，这些叠加的词向量就构成一个类似图片的二维“面板”数据。

以下列文本为例（已经分词）。

## 基于RNN/LSTM的文本分类模型

In [1]:
import pandas as pd
import numpy as np
import cntext 
import jieba

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
file = './nlp_data/douban_moview_reviews/DMSC.csv'
raw = pd.read_csv(file, header=0)
raw.head()

,ID,Movie_Name_EN,Movie_Name_CN,Crawl_Date,Number,Username,Date,Star,Comment,Like
0,0,Avengers Age of Ultron,复仇者联盟2,2017-01-22,1,然潘,2015-05-13,3,连奥创都知道整容要去韩国。,2404
1,1,Avengers Age of Ultron,复仇者联盟2,2017-01-22,2,更深的白色,2015-04-24,2,非常失望，剧本完全敷衍了事，主线剧情没突破大家可以理解，可所有的人物都缺乏动机，正邪之间、...,1231
2,2,Avengers Age of Ultron,复仇者联盟2,2017-01-22,3,有意识的贱民,2015-04-26,2,2015年度最失望作品。以为面面俱到，实则画蛇添足；以为主题深刻，实则老调重弹；以为推陈出...,1052
3,3,Avengers Age of Ultron,复仇者联盟2,2017-01-22,4,不老的李大爷耶,2015-04-23,4,《铁人2》中勾引钢铁侠，《妇联1》中勾引鹰眼，《美队2》中勾引美国队长，在《妇联2》中终于...,1045
4,4,Avengers Age of Ultron,复仇者联盟2,2017-01-22,5,ZephyrO,2015-04-22,2,虽然从头打到尾，但是真的很无聊啊。,723


In [3]:
documents = [raw.Comment[i] for i in range(raw.shape[0]) ]

In [4]:
tokenizer = cntext.cnTokenizer()
tokenizer.fit_on_cntexts(documents)
V = len(tokenizer.word_index) + 1
text_sequences=tokenizer.cntexts_to_sequences(documents)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\xieliang\AppData\Local\Temp\jieba.cache
Loading model cost 0.703 seconds.
Prefix dict has been built succesfully.


In [6]:
y = raw.Star

In [8]:
from keras.datasets import imdb
max_features=5000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

17465344/17464789 [==============================] - 3s 0us/step


In [11]:
x_train[:1]

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32])],
      dtype=object)

In [12]:

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

In [15]:
# set parameters:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [16]:
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 10s 401us/step - loss: 0.4062 - acc: 0.7953 - val_loss: 0.3109 - val_acc: 0.8676
Epoch 2/2
25000/25000 [==============================] - 6s 243us/step - loss: 0.2338 - acc: 0.9070 - val_loss: 0.2570 - val_acc: 0.8947
